# Лабораторная работа: Капрекар и неподвижные точки функции К(i)
* * * * * * * * * * * *
### Вычитание в соответствующей системе счисления
На вход поступает два массива list и основание системы счисления, происходит поразрядное вычитание

In [1]:
bad = ['A', 'B', 'C', 'D', 'E', 'F']

def substr(a, b, num_sys):
    res = []
    for i in range(len(b)):
        if (b[i] - a[i] >= 0): res.append(b[i] - a[i])
        else: 
            res.append(b[i] + num_sys - a[i])
            k = i + 1
            while ((b[k] == 0) and (k < len(b))):
                b[k] = b[k] + num_sys - 1
                k = k + 1
            #if (k == len(b):  - число < 0, в рамках наших вычислений такого не случится... можно добавить исключение
            b[k] = b[k] - 1
    ## для единственности найденных чисел сразу отсортируем его по убыванию
    res.reverse()
    for i in range(len(res)): 
        if(res[i] > 9): res[i] = bad[res[i] - 10]
    return ("".join(map(str,res)))

### Функция Капрекар. 
На вход поступает строка, в которой записано число, количество разрядов числа и система счисления  
На выходе функция возвращает строку - результат вычислений

In [2]:
def kaprekar(number, digit, num_sys):
    list_digits = [int(d, num_sys) for d in number]
    #while(digit > len(elem)):
    #    digits.append('0')
    list_digits.sort(reverse=True)
    max_sq = list_digits.copy()
    list_digits.sort()
    min_sq = list_digits.copy()
    #min_num = int("".join(map(str,min_sq))
    #max_num = int("".join(map(str,max_sq)))
    return substr(max_sq, min_sq, num_sys)

#### Введем переменные:  
list magicNumber содержит информацию о всех найденных неподвижных точках.  
            { строка-запись числа - ключ, система счисления и длина до магического числа - пара значений по ключу } 
  
list usedNumber вспомогательный, чтобы найти цикл.

list Cycles содержит информацию о всех найденных циклах.  
            [массив list - цикл, основание системы счисления, длина до цикла]

In [3]:
import math
magicNumber = {0: (math.inf, 2)}
usedNumber = []
Cycles = []

### Построение всех различных (для функции) чисел

Как будем строить число? Начнем с малого, заполним набор всех чисел для случая 2-разрядного числа с системе счисления по основанию 2. Это числа "11" и "10". Как далее происходит переход к большей разрядности или основанию С.С.?  

#### Разрядность:  
11 10       ->  (111, 110) + (100)  
111 110 100 -> (1111, 1110) + (1100) + (1000)   

#### Система счисления:  
11 10       + 22 21 20  
111 110 100 + 222 221 220 211 210 200


In [4]:
import numpy as np
batch_size = 10
# начальный массив возможных значений для d = 2, r = 2
start = ["11", "10"]

# строим таким образом, чтобы были только max значения (можно все min или премешанноые в виде 51423, 
# главное, чтобы одному набору цифр соответствовала только одна комбинация из них)

def add_d(arr, digit):
    tmp = []
    for i in arr:
        for k in range(3):
            if (int(i[digit - 2]) < k): break
            else: tmp.append(i + str(k))
    return tmp

def add_r(rez, number, digit, r):
    if (r > 10): number = number + bad[r - 11]
    else: number = number + (str(r - 1))
    for i in range (r):
        if (digit <= 1):
            rez.append(number)
            return rez
        else:
            add_r(rez, number, digit - 1, r - i)
    return rez

def in_Cycles(r):
    for arr in Cycles:
        if (arr[1] == r):
            for number in arr[0]:
                if (usedNumber[0] == number):
                    return True
    return False

### Process функция
На вход получает число, применяет к нему функцию капрекара, пока не найдется магическое число или процесс не зациклится

In [5]:
def process(number, d, r):
    usedNumber.clear()
    count = 0
    prev = "0"
    while True:
        prev = number
        usedNumber.append(number)
        number = kaprekar(number, d, r) 
        count = count + 1 
        if (int(number,r) == 0):
            break
        #else: 
            #print ("number is " + number)
        if (number == prev):
            if (magicNumber.get(number) == None):
                magicNumber[number] = (r, count)
                #print ("число " + number + " является неподвижной точкой: d = " + str(d) +" r = " + str(r) + "\n")
            elif (magicNumber[number][1] < count):
                magicNumber[number] = (r, count)
            break
        else:
            if(number in usedNumber): #найден цикл
                while (usedNumber[0] != number):
                    del usedNumber[0]
                if (not in_Cycles(r)): Cycles.append((usedNumber[:], r, count))
                break

In [6]:
for d in range(2,17):
    if (d > 2): start = add_d(start, d)
    for r in range(2,17):
        if (r > 2): 
            collection.extend(add_r([],"", d, r))
        else: collection = start.copy()
        #print ("d = ",d,"r = ",r)
        # 9 раз берем рандомные числа из collection, если их больше, чем batch_size, и находим магические числа или циклы
        # можно установить число побольше и подождать подольше =) 
        for n in range (10):
            if (len(collection) > batch_size):
                order = np.random.permutation(len(collection))[0 : min(d * 5, batch_size)]
                x = np.array(collection)[order]
            else:
                x = np.array(collection)
                
            for number in x:
                process(number, d, r)
            if (len(x) < batch_size):
                break

KeyboardInterrupt: 

In [7]:
magicNumber

{0: (inf, 2),
 '01': (2, 2),
 '13': (5, 4),
 '25': (8, 3),
 '37': (11, 5),
 '49': (14, 3),
 '011': (2, 2),
 '132': (4, 4),
 '253': (6, 5),
 '374': (8, 6),
 '495': (10, 7),
 '5B6': (12, 8),
 '6D7': (14, 9),
 '7F8': (16, 10),
 '1001': (2, 2),
 '0111': (2, 2),
 '3021': (4, 4),
 '3032': (5, 5),
 '6174': (10, 8),
 '92B6': (15, 3),
 '10101': (2, 2),
 '01111': (2, 2),
 '20211': (3, 5),
 '41532': (6, 7),
 '62853': (9, 3),
 '83B74': (12, 13),
 'A4E95': (15, 14),
 '101101': (2, 2),
 '110001': (2, 2),
 '011111': (2, 2),
 '310221': (4, 7),
 '213312': (4, 2),
 '330201': (4, 3),
 '420432': (6, 6),
 '325523': (6, 2),
 '530421': (6, 4),
 '640632': (8, 17),
 '437734': (8, 2),
 '631764': (10, 5),
 '549945': (10, 2),
 '951A74': (13, 21),
 '1011101': (2, 2),
 '1101001': (2, 2),
 '0111111': (2, 2),
 '2202101': (3, 3),
 '3203211': (4, 7),
 '6417532': (8, 5),
 '962B853': (12, 5),
 'C83FB74': (16, 8),
 '10111101': (2, 2),
 '11011001': (2, 2),
 '11100001': (2, 2),
 '01111111': (2, 2),
 '21022111': (3, 6),
 '31

In [8]:
print(Cycles)

[(['03', '21'], 4, 3), (['05', '41', '23'], 6, 4), (['07', '61', '43'], 8, 5), (['53', '17'], 9, 3), (['27', '45', '09', '81', '63'], 10, 6), (['65', '0B', 'A1', '83', '47', '29'], 12, 8), (['57', '1B', '93'], 13, 5), (['2B', '85'], 14, 3), (['0D', 'C1', 'A3', '67'], 14, 6), (['0F', 'E1', 'C3', '87'], 16, 5), (['4B', '69', '2D', 'A5'], 16, 6), (['022', '121'], 3, 3), (['242', '143'], 5, 5), (['363', '264'], 7, 5), (['484', '385'], 9, 5), (['5A5', '4A6'], 11, 5), (['6C6', '5C7'], 13, 8), (['7E7', '6E8'], 15, 7), (['1221', '1012'], 3, 4), (['2022', '1332'], 4, 3), (['4132', '3043', '3552', '3133', '1554', '4042'], 6, 8), (['5052', '5232', '3054'], 7, 5), (['3774', '4244', '1776', '6062', '6332'], 8, 8), (['3065', '6152', '5243'], 8, 5), (['7072', '7432', '5074'], 9, 7), (['5254', '3076', '7252'], 9, 6), (['7274', '5276', '5096', '9092', '9632'], 11, 7), (['7094', '9272', '7454', '3098', '9452'], 11, 7), (['3BB8', '8284', '6376'], 12, 6), (['7BB4', '7375', '4198', '8374', '5287', '6196'],

### Иллюстрация результатов анализа функции Капрекара

Финальный этап - наглядное демонстрирование результатов работы всей программы. Делать мы это будем с помощью таблицы, где строки отвечают количеству цифр в числе, а столбцы - системе счисления, в которой проделана операция вычитания.

In [9]:
import pandas as pd

result = np.zeros((15,15), dtype = list)

for k, v in magicNumber.items():
    if (v[0] == math.inf):
        continue
    row = len(k) - 2
    col = v[0] - 2
    if (result[row][col] == 0):
        result[row][col] = [1, v[1], 0, 0, 0]
    else:
        result[row][col][0] += 1
        if (result[row][col][1] < v[1]):
            result[row][col][1] = v[1]


In [10]:
for v in Cycles:
    row = len(v[0][0]) - 2
    col = v[1] - 2
    if (result[row][col] == 0):
        result[row][col] = [0, 0, 1, v[2], len(v[0])]
    elif (result[row][col][2] == 0):
        result[row][col][2] = 1
        result[row][col][3] = v[2]
        result[row][col][4] = len(v[0])
    else:
        result[row][col][2] += 1
        if (result[row][col][3] < v[2]):
            result[row][col][3] = v[2]
        if (result[row][col][4] < len(v[0])):
            result[row][col][4] = len(v[0])

In [11]:
table = pd.DataFrame(result, 
                     columns = [str(d) + " radix" for d in range(2, 17)] )
table

# я посчитала только 14-значные числа, можно дождаться и 16-значных, я сделаю это чуть позже
# такой формат записи в таблицу мне также кажется неуобным для чтения, потому что невполне очевидно значение 
# этих цифр в массиве list. (Постараюсь добавить строки вместо массивов)

# 1 число - количество магических чисел
# 2 число - максимальное количество шагов, за которое будет достигнуто магическое число
# 3 число - количество циклов 
# 4 число - максимальное количество -\\- будет достигнут цикл
# 5 число - наибольшая длина цикла

,2 radix,3 radix,4 radix,5 radix,6 radix,7 radix,8 radix,9 radix,10 radix,11 radix,12 radix,13 radix,14 radix,15 radix,16 radix
0,"[1, 2, 0, 0, 0]",0,"[0, 0, 1, 3, 2]","[1, 4, 0, 0, 0]","[0, 0, 1, 4, 3]",0,"[1, 3, 1, 5, 3]","[0, 0, 1, 3, 2]","[0, 0, 1, 6, 5]","[1, 5, 0, 0, 0]","[0, 0, 1, 8, 6]","[0, 0, 1, 5, 3]","[1, 3, 2, 6, 4]",0,"[0, 0, 2, 6, 4]"
1,"[1, 2, 0, 0, 0]","[0, 0, 1, 3, 2]","[1, 4, 0, 0, 0]","[0, 0, 1, 5, 2]","[1, 5, 0, 0, 0]","[0, 0, 1, 5, 2]","[1, 6, 0, 0, 0]","[0, 0, 1, 5, 2]","[1, 7, 0, 0, 0]","[0, 0, 1, 5, 2]","[1, 8, 0, 0, 0]","[0, 0, 1, 8, 2]","[1, 9, 0, 0, 0]","[0, 0, 1, 7, 2]","[1, 10, 0, 0, 0]"
2,"[2, 2, 0, 0, 0]","[0, 0, 1, 4, 2]","[1, 4, 1, 3, 2]","[1, 5, 0, 0, 0]","[0, 0, 1, 8, 6]","[0, 0, 1, 5, 3]","[0, 0, 2, 8, 5]","[0, 0, 2, 7, 3]","[1, 8, 0, 0, 0]","[0, 0, 2, 7, 5]","[0, 0, 2, 7, 6]","[0, 0, 3, 7, 6]","[0, 0, 1, 6, 3]","[1, 3, 6, 8, 4]","[0, 0, 4, 10, 6]"
3,"[2, 2, 0, 0, 0]","[1, 5, 0, 0, 0]","[0, 0, 1, 6, 2]","[0, 0, 1, 6, 4]","[1, 7, 1, 3, 2]","[0, 0, 1, 10, 5]","[0, 0, 2, 8, 4]","[1, 3, 1, 8, 5]","[0, 0, 3, 8, 4]","[0, 0, 1, 8, 4]","[1, 13, 1, 4, 2]","[0, 0, 1, 14, 4]","[0, 0, 2, 8, 4]","[1, 14, 0, 0, 0]","[0, 0, 2, 10, 4]"
4,"[3, 2, 0, 0, 0]","[0, 0, 1, 4, 3]","[3, 7, 0, 0, 0]","[0, 0, 1, 7, 5]","[3, 6, 1, 7, 3]","[0, 0, 1, 11, 6]","[2, 17, 1, 7, 3]","[0, 0, 1, 17, 14]","[2, 5, 1, 8, 7]","[0, 0, 1, 15, 4]","[0, 0, 1, 12, 10]","[1, 21, 0, 0, 0]","[0, 0, 3, 22, 12]","[0, 0, 2, 20, 9]","[0, 0, 5, 17, 5]"
5,"[3, 2, 0, 0, 0]","[1, 3, 1, 3, 2]","[1, 7, 0, 0, 0]","[0, 0, 1, 10, 4]","[0, 0, 1, 4, 2]","[0, 0, 1, 17, 6]","[1, 5, 2, 12, 7]","[0, 0, 1, 4, 2]","[0, 0, 1, 10, 8]","[0, 0, 2, 16, 14]","[1, 5, 1, 32, 9]","[0, 0, 3, 13, 12]","[0, 0, 1, 14, 8]","[0, 0, 2, 13, 10]","[1, 8, 2, 13, 11]"
6,"[4, 2, 0, 0, 0]","[1, 6, 0, 0, 0]","[3, 5, 1, 4, 3]","[0, 0, 1, 8, 6]","[2, 3, 3, 9, 7]","[0, 0, 1, 8, 6]","[1, 4, 2, 16, 3]","[1, 2, 1, 11, 4]","[1, 3, 2, 11, 7]","[1, 6, 2, 14, 7]","[1, 8, 1, 28, 24]","[1, 24, 1, 10, 4]","[1, 35, 3, 26, 8]","[1, 5, 3, 30, 22]","[0, 0, 6, 15, 8]"
7,"[4, 2, 0, 0, 0]","[1, 3, 2, 5, 3]","[3, 7, 0, 0, 0]","[1, 15, 0, 0, 0]","[0, 0, 1, 14, 2]","[0, 0, 1, 20, 11]","[0, 0, 3, 10, 5]","[1, 9, 1, 19, 12]","[1, 4, 1, 19, 14]","[1, 6, 1, 17, 7]","[0, 0, 3, 18, 13]","[0, 0, 3, 18, 5]","[0, 0, 1, 24, 8]","[1, 6, 2, 25, 8]","[1, 5, 3, 15, 8]"
8,"[5, 2, 0, 0, 0]","[1, 6, 1, 6, 4]","[5, 8, 0, 0, 0]","[0, 0, 1, 8, 4]","[4, 10, 3, 5, 2]","[0, 0, 3, 12, 2]","[1, 2, 4, 11, 3]","[1, 2, 2, 9, 5]","[1, 9, 5, 15, 7]","[0, 0, 4, 14, 11]","[0, 0, 3, 28, 5]","[1, 24, 3, 18, 9]","[0, 0, 6, 20, 6]","[0, 0, 1, 11, 6]","[0, 0, 12, 35, 8]"
9,"[5, 2, 0, 0, 0]","[2, 3, 3, 4, 3]","[3, 6, 1, 7, 3]","[1, 3, 1, 13, 3]","[2, 6, 1, 10, 2]","[1, 16, 0, 0, 0]","[0, 0, 3, 12, 6]","[0, 0, 1, 12, 6]","[1, 8, 2, 14, 8]","[0, 0, 3, 13, 10]","[1, 11, 2, 26, 21]","[0, 0, 1, 26, 14]","[0, 0, 3, 24, 10]","[0, 0, 3, 26, 11]","[1, 16, 3, 19, 9]"
